In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from imutils.video import VideoStream
import imutils
import time

In [ ]:


# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
lftObjpoints = [] # 3d point in real world space
lftImgpoints = [] # 2d points in image plane.
# Arrays to store object points and image points from all the images.
rhtObjpoints = [] # 3d point in real world space
rhtImgpoints = [] # 2d points in image plane.# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
lftObjpoints = [] # 3d point in real world space
lftImgpoints = [] # 2d points in image plane.
# Arrays to store object points and image points from all the images.
rhtObjpoints = [] # 3d point in real world space
rhtImgpoints = [] # 2d points in image plane.

# TODO: Use more stable identifiers
left = cv2.VideoCapture(1)
right = cv2.VideoCapture(2)

# dramatically lower resolution to avoid overloading usb
left.set(cv2.CAP_PROP_FRAME_WIDTH, 16)
left.set(cv2.CAP_PROP_FRAME_HEIGHT, 16)
right.set(cv2.CAP_PROP_FRAME_WIDTH, 16)
right.set(cv2.CAP_PROP_FRAME_HEIGHT, 16)

# Use MJPEG to avoid overloading the USB 2.0 bus at this resolution
left.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*"MJPG"))
right.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*"MJPG"))

# Grab both frames first, then retrieve to minimize latency between cameras
while True:
    
    if left.grab() and right.grab():
    
        _, leftFrame = left.retrieve()
        leftWidth, leftHeight = leftFrame.shape[:2]
        _, rightFrame = right.retrieve()
        rightWidth, rightHeight = rightFrame.shape[:2]

        # TODO: Calibrate the cameras and correct the images



        lftGray = cv2.cvtColor(leftFrame,cv2.COLOR_BGR2GRAY)
        rhtGray = cv2.cvtColor(rightFrame,cv2.COLOR_BGR2GRAY)

        # Find the chess board corners
        lftRet, lftCorners = cv2.findChessboardCorners(lftGray, (7,6),None)
        rhtRet, rhtCorners = cv2.findChessboardCorners(rhtGray, (7,6),None)

        # If found, add object points, image points (after refining them)
        if lftRet == True:
            lftObjpoints.append(objp)

            lftCorners2 = cv2.cornerSubPix(lftGray,lftCorners,(11,11),(-1,-1),criteria)
            lftImgpoints.append(lftCorners2)

            # Draw and display the corners
            leftFrame = cv2.drawChessboardCorners(leftFrame, (7,6), lftCorners2,lftRet)
               # show the output frame


                   # If found, add object points, image points (after refining them)
        if rhtRet == True:
            rhtObjpoints.append(objp)

            rhtCorners2 = cv2.cornerSubPix(lftGray,lftCorners,(11,11),(-1,-1),criteria)
            rhtImgpoints.append(rhtCorners2)

            # Draw and display the corners
            rightFrame = cv2.drawChessboardCorners(rightFrame, (7,6), rhtCorners2,rhtRet)
               # show the output frame






    
    
    
    cv2.imshow('left', leftFrame)
    cv2.imshow('right', rightFrame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

left.release()
right.release()
cv2.destroyAllWindows()


In [2]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

vc = cv2.VideoCapture(1)
vc.set(cv2.CAP_PROP_FRAME_WIDTH, 16)
vc.set(cv2.CAP_PROP_FRAME_HEIGHT, 16)

vc.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*"MJPG"))


while True:
    ret, img = vc.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)
        img = cv2.drawChessboardCorners(img, (7,6), corners2,ret)

    cv2.imshow('right', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cv2.destroyAllWindows()



   

In [3]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

#Save parameters into numpy file
np.save("./lft_camera_params/imgpoints", imgpoints)
np.save("./lft_camera_params/ret", ret)
np.save("./lft_camera_params/K", mtx)
np.save("./lft_camera_params/dist", dist)
np.save("./lft_camera_params/rvecs", rvecs)
np.save("./lft_camera_params/tvecs", tvecs)

In [7]:
stereocalibration_criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 100, 1e-5)
stereocalibration_flags = cv2.CALIB_FIX_INTRINSIC
## need to record a video from stero cameras at the same time and use that for callibration.
stereocalibration_retval, cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, R, T, E, F = cv2.stereoCalibrate(objpoints, np.load("./lft_camera_params/imgpoints.npy"), np.load("./right_camera_params/imgpoints.npy"), 
                                                                                                                    np.load("./lft_camera_params/K.npy"), np.load("./lft_camera_params/dist.npy"), 
                                                                                                                    np.load("./right_camera_params/K.npy"), np.load("./right_camera_params/dist.npy"), 
                                                                                                                    gray.shape[::-1], 
                                                                                                                   criteria = stereocalibration_criteria, flags = stereocalibration_flags)

error: OpenCV(4.3.0) C:\projects\opencv-python\opencv\modules\calib3d\src\calibration.cpp:3336: error: (-2:Unspecified error) in function 'void __cdecl cv::collectCalibrationData(const class cv::_InputArray &,const class cv::_InputArray &,const class cv::_InputArray &,int,class cv::Mat &,class cv::Mat &,class cv::Mat *,class cv::Mat &)'
>  (expected: 'nimages == (int)imagePoints2.total()'), where
>     'nimages' is 36
> must be equal to
>     '(int)imagePoints2.total()' is 28


In [ ]:
vs = VideoStream(src=1).start()

while True:
    
    img = vs.read()
#     plt.imshow(img)
#     plt.show()
    
        
    h,  w = img.shape[:2]
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
    
    
    # undistort
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)


    # show the output frame
    cv2.imshow("Frame", img)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

    
cv2.destroyAllWindows()


In [ ]:
import numpy as np
import argparse
import cv2
from imutils.video import VideoStream
from imutils.video import FPS
import time
import imutils
from scipy.spatial import distance
from typing import NamedTuple
import collections

In [ ]:
vs = VideoStream(src=1).start()

while True:
   
    frame = vs.read()
    
    
       # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
    # update the FPS counter


cv2.destroyAllWindows()
vs.stop()